In [12]:
import xgboost as xgb
import pandas as pd
import numpy as np
import sklearn
import pickle

In [13]:
input_data = pd.read_csv('voice.csv')
input_data = input_data.sample(frac=1)  
gender = {'male' : 0, 'female' : 1}
input_data['label'] = input_data['label'].map(gender)
cols = [c for c in input_data.columns if c not in ['label']]

In [14]:
input_data.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
807,0.180360,0.053766,0.189474,0.141895,0.213474,0.071579,1.473165,4.899974,0.918149,0.395065,...,0.180360,0.132277,0.047013,0.279070,0.968568,0.023438,4.148438,4.125000,0.128314,0
1593,0.184250,0.069538,0.204847,0.180113,0.227361,0.047248,2.780535,12.557136,0.909607,0.526946,...,0.184250,0.143819,0.015764,0.271186,0.222982,0.054688,1.015625,0.960938,0.206433,1
424,0.204982,0.061316,0.234010,0.133237,0.254976,0.121739,1.850012,5.706426,0.867338,0.283762,...,0.204982,0.120147,0.018161,0.231884,0.408381,0.148438,0.796875,0.648438,0.312048,0
2228,0.134673,0.070310,0.148235,0.066275,0.190196,0.123922,1.196346,5.295374,0.962069,0.665675,...,0.134673,0.152538,0.047431,0.279070,1.372500,0.023438,7.359375,7.335938,0.125617,1
1240,0.172130,0.052122,0.169406,0.137443,0.207443,0.070000,1.089934,3.996472,0.932482,0.466363,...,0.172130,0.129677,0.047105,0.279070,0.925118,0.023438,4.382812,4.359375,0.131541,0


In [15]:
train = input_data.iloc[0 :2400]
test = input_data.iloc[2400 : ]
test_label = test['label']
test_label = np.array(test_label).reshape([-1 , 1])
del(test['label'])

In [16]:
def xgb_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'log_loss', sklearn.metrics.log_loss(labels, preds)

In [17]:
fold = 1
for i in range(fold):
    params = {
        'eta': 0.01, #use 0.002
        'max_depth': 5,
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'seed': i,
        'silent': True
    }
    #x1, x2, y1, y2 = sklearn.model_selection.train_test_split(train[cols], train['label'], test_size=0.3, random_state=i)
    x1 = train[cols][0:2000]
    x2 = train[cols][2000 : 2400]
    y1 = train['label'][0:2000]
    y2 = train['label'][2000 : 2400]
    watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]
    model = xgb.train(params, xgb.DMatrix(x1, y1), 1500,  watchlist, feval=xgb_score, maximize=False, verbose_eval=50, early_stopping_rounds=50) #use 1500
    if i != 0:
        pred += model.predict(xgb.DMatrix(test[cols]), ntree_limit=model.best_ntree_limit)
    else:
        pred = model.predict(xgb.DMatrix(test[cols]), ntree_limit=model.best_ntree_limit)
        
pred /= fold
pre_label = np.zeros([pred.shape[0], 1])
for i in range(pred.shape[0]):
    if pred[i] >= 0.5:
        pre_label[i] = 1
    else:
        pre_label[i] = 0

acc = np.mean(np.equal(pre_label, test_label).astype(np.float))
print("测试集正确率： ", acc)


[0]	train-log_loss:0.683795	valid-log_loss:0.684025
Multiple eval metrics have been passed: 'valid-log_loss' will be used for early stopping.

Will train until valid-log_loss hasn't improved in 50 rounds.
[50]	train-log_loss:0.37519	valid-log_loss:0.386187
[100]	train-log_loss:0.224466	valid-log_loss:0.24541
[150]	train-log_loss:0.141292	valid-log_loss:0.167645
[200]	train-log_loss:0.092068	valid-log_loss:0.122265
[250]	train-log_loss:0.063022	valid-log_loss:0.096878
[300]	train-log_loss:0.045153	valid-log_loss:0.081632
[350]	train-log_loss:0.033377	valid-log_loss:0.074542
[400]	train-log_loss:0.025643	valid-log_loss:0.071061
[450]	train-log_loss:0.020681	valid-log_loss:0.069805
[500]	train-log_loss:0.017202	valid-log_loss:0.069796
[550]	train-log_loss:0.014621	valid-log_loss:0.069379
[600]	train-log_loss:0.012723	valid-log_loss:0.069392
[650]	train-log_loss:0.01127	valid-log_loss:0.068705
[700]	train-log_loss:0.010108	valid-log_loss:0.068607
Stopping. Best iteration:
[687]	train-log_l

In [18]:
model_save = open('model.pkl', 'wb')
pickle.dump(model, model_save)
model_save.close()